<a href="https://colab.research.google.com/github/Siwenli0615/General/blob/main/Laboration2%20FactionWar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import random
import matplotlib.pyplot as plt
import time

class Faction:


    def __init__(self,name:str,apples:int,oranges:int,bananas:int,strawberries:int):
      # använder en for-loop för att granska alla tal-baserad argumenter (frukterna)
      fruits = {"apples":apples,"oranges":oranges,"bananas":bananas,"strawberries":strawberries}
      for key,value in fruits.items():
        if not isinstance(value,int):
          raise TypeError(f'{key} needs to be a number')
        if not (0<=value<=200):
          raise ValueError(f"The limit of amount of {key} can only be between 0-200 ({value} provided)")
      if not isinstance(name,str):
        raise TypeError("Name needs to be a string")

      self.__name = name
      self.__apples = apples
      self.__oranges = oranges
      self.__bananas = bananas
      self.__strawberries = strawberries

    @property
    def name(self):
      return self.__name
    @property
    def apples(self):
      return self.__apples
    @property
    def oranges(self):
      return self.__oranges
    @property
    def bananas(self):
      return self.__bananas
    @property
    def strawberries(self):
      return self.__strawberries




    def __repr__(self)->str:
      #visar exakt samma mening som efterfrågat i upppgiften
      text = f"faction(name='{self.__name}',apples={self.__apples}, oranges={self.__oranges}, bananas={self.__bananas}, strawberries={self.__strawberries})"
      return text




    def inventory(self)->int:
      total_fruits = self.__apples + self.__oranges + self.__bananas + self.__strawberries
      return total_fruits




    def status(self):
      total_frukt = self.inventory()
      if total_frukt <= 150:
        print(f"This aint nothin but a scratch! {self.__name} might be struggling for the moment, but will be back with vengance! (strength:{total_frukt})")
      elif (150<total_frukt<=500):
        print(f"The proud {self.__name} is VERY eager to toss some apples! Where are the opponents?! (strength:{total_frukt})")
      elif 500<total_frukt:
        print(f"{self.__name} is GODLIKE! Their position is super dominant! (strength:{total_frukt})")
      elif total_frukt == 0:
        print(f"{self.name} is eliminated.")



    def alter_stock_value(self,fruit:str,stock:int):
      possible_fruits = ("apples","oranges","bananas","strawberries") # skall endast acceptera mellan de 4 sorter frukter
      fruit = fruit.lower().strip()                  # ta bort risken för stora bokstav och mellanslag
      if not isinstance(fruit,str):                  # samma kontroll som initialiseringen
        raise TypeError("Fruit should be in text (string)")
      elif fruit not in possible_fruits:
        raise ValueError(f"{fruit} not found in inventory (apples/oranges/bananas/strawberries)")
      if not isinstance(stock,int):
        raise TypeError(f"The amount of {fruit} need to be a number (integer)")
      elif not (0<=stock<=200):
        raise ValueError(f"The amount of {fruit} need to be between 0-200")
      attribute_name = f'_Faction__{fruit}'             # skulle fungera med 4 if-satser, men jag tyckte
      setattr(self,attribute_name,stock)              # att det är bra att lära sig en metod som kan ta många attributer



    def visualize_resources(self):
      # mitt mål är att plotta olika frukter mot deras antal:
      fruits = ['apples','oranges','bananas','strawberries']
      fruit_amounts = [self.__apples,self.__oranges,self.__bananas,self.__strawberries]

      # olika färger beroende på totalen på olika frukter
      colors = []
      max_value = 200
      for amount in fruit_amounts:
        if amount >= 100:
            colors.append('green')
        elif 50 < amount <= 100:
            colors.append('orange')
        else:
            colors.append('red')

      x_values = fruits
      y_values = fruit_amounts
      # Vill ha snyggare utseende, därför hittade nedan lösningar på nätet:
      # Create an array of 200s for shadow bars
      shadow_values = [max_value] * len(fruits)
      # Create the figure and axis objects
      fig, ax = plt.subplots()
      # Plot the shadow bars with constant value 200
      ax.barh(x_values, shadow_values, color='lightgray', label='Max Amount (200)', alpha=0.5)

      # Plot the actual fruit amounts in front of the shadow bars
      bars = ax.barh(x_values, y_values, color=colors, label='Current Amount')

      plt.title(f"Resourses for faction {self.name}")
      plt.xlabel("Resourses")

      #showing the value of each fruit:
      for i, bar in enumerate(bars):
          value = y_values[i]
          ax.text(value, i, str(value), va='center')
      # Show the chart
      plt.show()



    def organical_growth(self):
      # Tar fram nuvarande namn och antal på frukterna
      fruits = ['apples','oranges','bananas','strawberries']
      fruit_amounts = [self.__apples,self.__oranges,self.__bananas,self.__strawberries]
      # lade till lite extra: om man har 0 på allt, då räknas man som utslagen
      if self.inventory() ==0:
        return (f"{self.name} is eliminated and can not gain any more growth")

      # självaste tillväxten: beror på
      # 1. om inte full (200) men full efter tillväxt,
      # 2. inte full och inte full efter tillväxt samt
      # 3. full i början
      for fruit,amount in zip(fruits,fruit_amounts):
        growth = random.randint(1,5)
        if amount < 200:            # Tillväxt skall endast var när den är under 200
          new_amount = amount + growth
          if new_amount >200:         # Om det blir över 200 efter tillväxten, ställ om till 200
            amount = 200
            print(f"Glorious {self.name} has harvested {growth} {fruit}, to a new total of 200! (max limit reached)")
          else:
            amount = new_amount       # annars vanlig tillväxt
            print(f"Glorious {self.name} has harvested {growth} {fruit}, to a new total of {amount}!")


        elif amount == 200:           # om full (200) från början:
          print(f"Glorious {self.name} is already at maximum capacity for {fruit}!")
        self.alter_stock_value(fruit,amount)


    def __mul__(self, other):
      # grundvärde från uppgiften
      success_rate = 70   # 70% chance for a successful attack
      pillage_rate = 20   # 20% resource transfers to the attacker if win
      crit_rate = 10     # 10% chance for crit
      pillage_rate_crit = 40 # 40% resource transfers if win with crit

      # ladda ner befintlig värde av self(attacker) och other(defender)
      fruits = ['apples','oranges','bananas','strawberries']
      self_fruit_amounts = [self.__apples,self.__oranges,self.__bananas,self.__strawberries]
      other_fruit_amounts = [other.__apples,other.__oranges,other.__bananas,other.__strawberries]

      # Special effekt
      print(f"{self.name} is attacking {other.name}!")
      print(f"The attack starts in")
      time.sleep(1)
      print("3")
      time.sleep(1)
      print("2")
      time.sleep(1)
      print("1")
      time.sleep(1)
      print(f"{self.name} is entering {other.name}'s territory!")
      time.sleep(1)
      print(f"the battle starts!")
      time.sleep(3)

      # ifall försvarare är svag (total frukt<5), ett alternativ för final blow erbjuds:
      if other.inventory == 0:
        print(f"{other.name} is already eliminated.")
      if (0< other.inventory() <= 5):
        final_blow = input(f"There is no GLORY in kicking down on an opponent already brought to its knees, make a final blow? (y/n)").lower().strip()
        if final_blow == "y":
          sucessRate = 50       # eget påhitt, kan ändras efter behov.
          if random.randint(1,100)<sucessRate:
            print(f"{self.name} made a successful final blow and stole all {other.inventory()} fruits {other.name} had left, {other.name} is now eliminated")
            # beräkna looten (tar allt från den eliminerad vid lyckad final blow)
            result_self = []
            for amount_self,amount_other in zip(self_fruit_amounts,other_fruit_amounts):
              if amount_self < 200:
                new_amount_self = amount_self + amount_other
                if new_amount_self > 200:
                  amount_self = 200
                else:
                  amount_self = round(new_amount_self)
                result_self.append(amount_self)
              elif amount_self == 200:
                result_self.append(200)

            for fruit in fruits:        # set defender resource to zero
              other.alter_stock_value(fruit,0)
            return
          else:
            print(f"{other.name} survived the blow and the game continues")
            return
        elif final_blow == 'n':
          print(f"{self.name} chose to spare the faith of {other.name}")
          return
        else:
          print(f"Input error: please choose between y and n: ")

      # detalj för en icke-eliminerande attack:
      # 1. om attacken lyckas och är critical
      outcome = random.randint(1,100)
      if outcome < success_rate:      # success_rate från grundvärde i början av def.
        # om den lyckad attacken är critical (10% chans):
        if random.randint(1,100) <= crit_rate:
          print(f"The attack was successful and {self.name} made a critial attack!")
          # beräkna loot critical:
          result_self = []
          other_new_amount = [x*(100-pillage_rate_crit)/100 for x in other_fruit_amounts]

          for amount_self,amount_other in zip(self_fruit_amounts,other_fruit_amounts):
            if amount_self < 200:
              new_amount_self = amount_self + amount_other*pillage_rate_crit/100
              if new_amount_self > 200:
                amount_self = 200
              else:
                amount_self = round(new_amount_self)
              result_self.append(amount_self)
            elif amount_self == 200:
              result_self.append(200)

        # 2. om attacken lyckas men ingen crit
        else:
          print(f"The attack was successful.")
          # beräkna loot vanlig attack:
          result_self = []
          other_new_amount = [x*(100-pillage_rate)/100 for x in other_fruit_amounts]

          for amount_self,amount_other in zip(self_fruit_amounts,other_fruit_amounts):
            if amount_self < 200:
              new_amount_self = amount_self + amount_other*pillage_rate/100
              if new_amount_self > 200:
                amount_self = 200
              else:
                amount_self = round(new_amount_self)
              result_self.append(amount_self)
            elif amount_self == 200:
              result_self.append(200)

        # beräkna vinnarens loot (attackerarens riktig vinst):
        loot = [round((new-old)) for new,old in zip(result_self,self_fruit_amounts)]
        fruits = ['apples','oranges','bananas','strawberries']
        print(f"{self.name} looted {loot[0]} {fruits[0]},{loot[1]} {fruits[1]},{loot[2]} {fruits[2]} and {loot[3]} {fruits[3]}")
        # updatera nya värde av frukterna för self och other:
        for fruit,value in zip(fruits,result_self):
          self.alter_stock_value(fruit,round(value))

        for fruit,value in zip(fruits,other_new_amount):
          other.alter_stock_value(fruit,round(value))
        self.visualize_resources()

      # 3. om attacken misslyckas.
      else:
        print(f"{other.name}have defended skillfully, {self.name} failed in the attack and did not gain anything from {other.name}")





In [26]:
fire_nation = Faction('Fire Nation', 100, 80, 20, 100)
earth_kingdom = Faction('Earth Kingdom', 0, 0, 0, 0)
water_tribe = Faction('Water Tribe', 100, 100, 100, 100)
air_nomads = Faction('Air Nomads', 100, 100, 100, 100)

#print(fire_nation)
#print(earth_kingdom)
#print(water_tribe)
#print(air_nomads)

#pancake_cult = Faction('Pancake Cult', 30, 20, 240, 160)
#cupcake_assassins = Faction('Cupcake Assassins', 1, 1, 1, -10)
#milk_drinkers = Faction(['Milk Drinkers'], 30, 20, 150, 160)


#fire_nation.inventory()
#fire_nation.status()
#fire_nation.alter_stock_value("apples",150)
#print(fire_nation)
#fire_nation.visualize_resources()
#water_tribe.organical_growth()
#fire_nation * water_tribe